# Data Generation Needed for ACR/SWI Paper

In [2]:
#-------------------------- Standard Imports --------------------------#
%reload_ext autoreload
%autoreload 2
import kdephys as kde
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import acr
import os
plt.style.use('fast')
plt.style.use('/home/kdriessen/gh_master/kdephys/kdephys/plot/acr_plots.mplstyle')

# ---------------------------- EXTRAS --------------------------------#
from acr.utils import swi_subs_exps, sub_probe_locations, sub_exp_types
import pub_funcs as pf

In [4]:
# IMPORTANT INFORMAION
# --------------------

# Check the IMPORTANT PARAMETERS section of pub_funcs.py to ensure that the rebound length and relative state are as they should be.

# Bandpower - During Sleep Rebound

In [5]:
# NOTES:
# ------

# We take as the start of the rebound the first time that NREM occurs after the stimulation has ended.
# Then we simply add the variable REBOUND_LENGTH to that start time to get the end of the rebound.
# So, subjects may have differing amounts of NREM in this initial 1 hr following the start of the sleep rebound. 

In [7]:
# Note that as currently configured, this does NOT reprocess subject-experiment combinations that are already in the dataframe. 
# If this needs to be done, call the function with reprocess_existing=True, for the particular subject-experiment combination
for subject in swi_subs_exps:
    for exp in swi_subs_exps[subject]:
        pf.get_rebound_df(subject, exp)

swi-bl has no channel 7
swi-bl has no channel 10
swi-bl has no channel 13
swi has no channel 7
swi has no channel 10
swi has no channel 13
swi-bl has no channel 7
swi-bl has no channel 10
swi-bl has no channel 13
swi has no channel 7
swi has no channel 10
swi has no channel 13
swisin-bl has no channel 7
swisin-bl has no channel 10
swisin-bl has no channel 13
swisin has no channel 7
swisin has no channel 10
swisin has no channel 13
swisin-bl has no channel 7
swisin-bl has no channel 10
swisin-bl has no channel 13
swisin has no channel 7
swisin has no channel 10
swisin has no channel 13


# Bandpower - During Experimental Manipulation 

In [8]:
for subject in swi_subs_exps:
    for exp in swi_subs_exps[subject]:
        pf.get_stim_df(subject, exp)

swi-bl has no channel 7
swi-bl has no channel 10
swi-bl has no channel 13
swi has no channel 7
swi has no channel 10
swi has no channel 13
swi-bl has no channel 7
swi-bl has no channel 10
swi-bl has no channel 13
swi has no channel 7
swi has no channel 10
swi has no channel 13
swisin-bl has no channel 7
swisin-bl has no channel 10
swisin-bl has no channel 13
swisin has no channel 7
swisin has no channel 10
swisin has no channel 13
swisin-bl has no channel 7
swisin-bl has no channel 10
swisin-bl has no channel 13
swisin has no channel 7
swisin has no channel 10
swisin has no channel 13
